# Simulation and optimisation of pressure distribution in a combustion chamber

This exercise is concerned with the simulation and optimisation of pressure distribution within a combustion chamber to ensure efficient fuel combustion and enhance engine performance. The design and optimisation of pressure distribution are crucial for achieving a uniform distribution, and this analysis involves simulating how various burner configurations affect the pressure distribution. The study utilises a two-dimensional model that balances computational efficiency with accuracy, making it applicable to various engineering fields where combustion efficiency is critical.

## Problem formulation

To analyse the pressure distribution within a combustion chamber, a mathematical model is developed based on fundamental principles of fluid dynamics. This formulation encapsulates the effects of burner configurations on pressure distribution and provides a basis for optimising the chamber's design.

### Governing equations

The pressure distribution within the combustion chamber is governed by the Poisson equation, describing pressure variations due to burner sources:

$$
\nabla^2 P(x,y) = -\frac{f(x,y)}{k},
$$

-   $P(x,y)$ is the pressure field at spatial coordinates $(x,y)$ $\left[\text{Pa}\right]$,
-   $f(x,y)$ represents the pressure source function from burners $\left[\text{Pa}/\text{m}^2\right]$,
-   $k$ is a proportionality constant related to fluid properties.

In Cartesian coordinates, the equation expands to:

$$
\frac{\partial^2 P}{\partial x^2} + \frac{\partial^2 P}{\partial y^2} = -\frac{f(x,y)}{k}
$$

#### Initial and boundary conditions

The problem is solved under specific boundary and initial conditions, which influence the overall pressure distribution.

1. **Boundary conditions**

    - Dirichlet condition (fixed pressure at chamber walls):
        $$
        P(x,y) = P_0 \quad \text{for} \quad (x,y) \in \partial \Omega
        $$
    - Neumann condition (zero normal pressure gradient at exits):
        $$
        \frac{\partial P}{\partial n} = 0 \quad \text{at outlet boundaries}
        $$

2. **Initial condition**
    - The pressure is assumed to be **uniform** throughout the chamber before the burners are activated:
        $$
        P(x, y, t=0) = P_{\text{ambient}}
        $$

### Heat source model for burners

The burner heat input is modelled as a Gaussian distribution, representing spatial heat injection:

$$
f(x,y) = \sum_{i=1}^{N} A_i \exp \left( -\frac{(x - x_i)^2 + (y - y_i)^2}{\sigma^2} \right),
$$

where:

-   $(x_i, y_i)$ are the burner locations,
-   $A_i$ is the burner intensity $\left[\text{Pa}/\text{m}^2\right]$ (per semplicità $A_i = 1\ \forall i$),
-   $\sigma$ is the spread parameter, controlling burner heat dissipation.

This formulation assumes each burner contributes additively to the pressure field, with localised effects decaying exponentially away from the burner centre.

### Numerical Solution

#### Finite Difference Method (FDM) discretisation

Using a structured grid with spacing $h$, the Poisson equation is discretised as (after rearranging for iterative solution):

$$
P_{i,j}^{(n+1)} = \frac{P_{i+1,j}^{(n)} + P_{i-1,j}^{(n)} + P_{i,j+1}^{(n)} + P_{i,j-1}^{(n)} - h^2 f_{i,j} / k}{4}.
$$

#### Iterative solvers and convergence criteria

-   Gauss-Seidel method: Solves iteratively, updating grid points in place.
-   Successive over-relaxation (SOR): Accelerates convergence using a relaxation parameter $\omega$.
-   Multigrid methods: Applied for large-scale simulations to enhance computational efficiency.

The stopping criterion is based on the residual error:

$$
\max \left| P^{(n+1)} - P^{(n)} \right| < \epsilon.
$$

where $\epsilon$ is a user-defined convergence threshold.

### Optimisation of burner configuration

#### Objective function: pressure uniformity

To achieve an optimal pressure distribution, we define a cost function based on pressure variance:

$$
J = \sum_{i,j} (P_{i,j} - \overline{P})^2.
$$

where $\overline{P}$ is the mean pressure value across the combustion chamber. This ensures the pressure values across the chamber are as close as possible to each other.

#### Optimisation technique: gradient-based optimisation

Both the burner positions $(x_i, y_i)$ and their intensities $A_i$ are adjusted iteratively to minimise the cost function:

$$
x_i^{(n+1)} = x_i^{(n)} - \alpha \frac{\partial J}{\partial x_i}, \quad y_i^{(n+1)} = y_i^{(n)} - \alpha \frac{\partial J}{\partial y_i}, \quad A_i^{(n+1)} = A_i^{(n)} - \alpha \frac{\partial J}{\partial A_i}.
$$

where $\alpha$ is the learning rate. The optimisation process follows a gradient-based approach, updating parameters based on the sensitivity of the cost function to changes in burner positions and intensities.

### Model assumptions

1. The combustion chamber is modelled as a two-dimensional system, assuming negligible variations along the third dimension.  
2. The pressure distribution is assumed to reach a steady-state, meaning time-dependent effects are not considered.  
3. The combustion chamber is treated as a uniform medium with no spatial variations in material properties.  
4. The gas within the chamber follows the assumptions of an ideal gas, where compressibility effects are not explicitly considered.  
5. The chamber walls are assumed to have fixed boundary conditions, ensuring a well-posed numerical solution.  
6. The heat sources (burners) are modelled using a Gaussian distribution to represent localised pressure contributions.  


## Implementation

The implementation presented herein pertains to solving the pressure distribution problem within a combustion chamber. The solution employs the Finite Difference Method (FDM) to discretise the governing equations. The solution is iteratively computed across the defined grid, adhering to the specified boundary and initial conditions.

### Parameter definition and initialisation

First, the domain parameters, boundary conditions, burner configuration, and initial conditions are defined as follows.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Ensure interactive plots in Jupyter Notebook
%matplotlib widget
plt.close("all")

# Seed for reproducibility
# np.random.seed(37)

# Domain parameters
Lx, Ly = 10.0, 10.0  # Dimensions of the combustion chamber (m)
Nx, Ny = 50, 50  # Number of grid points in x and y directions
h = Lx / (Nx - 1)  # Grid spacing (assumed same in x and y)

# Pressure parameters
P0 = 90.0  # Fixed pressure at chamber walls (Pa)
P_ambient = 1.0  # Initial uniform pressure (Pa)

# Burner parameters
num_burners = 4  # Number of burners
sigma = 1.0  # Spread parameter for the Gaussian burner model
k_const = 1.0  # Proportionality constant in the Poisson equation

# Generate initial burner configuration: (x, y) for each burner
burners_init = [
    (np.random.uniform(0, Lx), np.random.uniform(0, Ly)) for _ in range(num_burners)
]

# Create spatial grid for evaluation
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)


### Burner source model

The burner pressure input is modelled as a Gaussian distribution representing localised pressure injection. The contributions from all burners are summed, and the model is incorporated into the Poisson equation.

In [ ]:
def calculate_burner_source(X, Y, burners, sigma=sigma, k_const=k_const):
    """
    Calculate the heat source term for the Poisson equation based on Gaussian burners.

    Parameters
    ----------
    X : np.ndarray
        Array of x-coordinates.
    Y : np.ndarray
        Array of y-coordinates.
    burners : list
        List of burners, where each burner is a tuple (x, y).
    sigma : float
        Spread parameter for the Gaussian burner model.
    k_const : float
        Proportionality constant in the Poisson equation.

    Returns
    -------
    np.ndarray
        Heat source term for the Poisson equation.
    """
    # Initialise heat source term
    f = np.zeros_like(X)

    # Add contribution from each burner
    for xb, yb in burners:
        f += np.exp(-((X - xb) ** 2 + (Y - yb) ** 2) / sigma**2)

    # Negative sign as per the Poisson equation
    return -f / k_const


### Pressure update

The pressure field is updated using the finite difference discretisation of the Poisson equation.

In [ ]:
def update_pressure(P, f, h, k_const=k_const):
    """
    Update the pressure field using the Poisson equation.

    Parameters
    ----------
    P : np.ndarray
        Pressure field at the current time step.
    f : np.ndarray
        Heat source term for the Poisson equation.
    h : float
        Grid spacing.
    k_const : float
        Proportionality constant in the Poisson equation.

    Returns
    -------
    np.ndarray
        Pressure field at the next time step.
    """
    # Create a copy of the pressure field
    P_new = P.copy()

    # Update pressure field using the Poisson equation
    for i in range(1, Nx - 1):
        for j in range(1, Ny - 1):
            P_new[i, j] = (
                P[i + 1, j]
                + P[i - 1, j]
                + P[i, j + 1]
                + P[i, j - 1]
                - h**2 * f[i, j] / k_const
            ) / 4

    return P_new


### Boundary conditions

The specified boundary conditions are applied to the pressure field to ensure accurate simulation of the chamber's physical constraints.

In [ ]:
def apply_boundary_conditions(P, P0):
    """
    Apply the specified boundary conditions to the pressure field.

    Parameters
    ----------
    P : np.ndarray
        Pressure field.
    P0 : float
        Pressure value at the boundaries.

    Returns
    -------
    np.ndarray
        Pressure field with boundary conditions applied.
    """
    P[0, :] = P0  # Left boundary
    P[-1, :] = P0  # Right boundary
    P[:, 0] = P0  # Bottom boundary
    P[:, -1] = P0  # Top boundary

    return P


### Iterative solver

An iterative Gauss-Seidel method is used to solve the Poisson equation until the solution converges. The convergence criterion is based on the maximum absolute difference between successive iterations.

In [ ]:
def solve_poisson(P, f, h, tol=1e-5, max_iter=5000):
    """
    Solve the Poisson equation for pressure using the Jacobi iterative method.

    Parameters
    ----------
    P : np.ndarray
        Initial guess for the pressure field.
    f : np.ndarray
        Heat source term for the Poisson equation.
    h : float
        Grid spacing.
    tol : float, optional
        Error tolerance for convergence. The default is 1e-5.
    max_iter : int, optional
        Maximum number of iterations. The default is 10000.

    Returns
    -------
    np.ndarray
        Pressure field after solving the Poisson equation.
    """
    for it in range(max_iter):
        # Copy the pressure field and update it
        P_old = P.copy()

        # Update the pressure field
        P = update_pressure(P, f, h)
        P = apply_boundary_conditions(P, P0)

        # Check for convergence
        if np.linalg.norm(P - P_old) < tol:
            break

    return P


## Results

### Complete simulation

The simulation is conducted iteratively to observe the pressure distribution within the combustion chamber until the steady-state is reached. At each iteration, the pressure field is updated based on the discretised Poisson equation and boundary conditions. 

In [ ]:
def solve_pressure_distribution(burners):
    """
    Computes the pressure distribution by solving the Poisson equation for a given burner configuration.

    Parameters
    ----------
    burners : list
        List of burners, where each burner is a tuple (x, y).

    Returns
    -------
    np.ndarray
        Pressure distribution in the combustion chamber.
    """
    # Calculate the source term for the Poisson equation
    f = calculate_burner_source(X, Y, burners)

    # Initialise the pressure field
    P = np.full((Nx, Ny), P_ambient)
    P = apply_boundary_conditions(P, P0)

    # Solve the Poisson equation to obtain the pressure field
    P = solve_poisson(P, f, h)

    return P

### Optimisation of burner configuration

Since `scipy.optimize.minimize` works with a 1D array, the helper functions convert the $(x,y)$ tuples into a flat array before optimisation and restore them afterward.

In [ ]:
def flatten_burners(burners):
    """Convert a structured array of burners into a flat array."""
    return np.array(burners).flatten()


def unflatten_burners(flat_burners, num_burners):
    """Convert flat array of burners to structured array."""
    return flat_burners.reshape(num_burners, 2).tolist()


#### Objective function: pressure uniformity

The cost function is defined to minimise the variance of the pressure field in order to achieve a pressure that is as uniform as possible across the combustion chamber.

In [ ]:
def objective_function(burners):
    """
    Compute source term and solve the Poisson equation for a given burner configuration.

    Parameters
    ----------
    burners : np.ndarray
        Array of burner parameters (x, y).

    Returns
    -------
    float
        Cost function value (pressure variance).
    np.ndarray
        Pressure field after solving the Poisson equation
    """
    # Parse the burner configuration using the helper function
    if burners.ndim == 1:
        burners = unflatten_burners(burners, len(burners) // 2)

    P = solve_pressure_distribution(burners)

    # Cost function: minimise the variance of the pressure field to achieve uniformity
    cost = np.var(P)

    return cost

### Visualisation of results


#### Initial pressure distribution

The initial pressure distribution is obtained using a randomly generated burner configuration. Pressure variations are influenced by the arbitrary placement and intensities of the burners, leading to non-uniform regions across the combustion chamber. Higher pressure zones appear near the initial burner positions, with gradients extending towards the boundaries.

In [ ]:
# Print initial burner configuration
print("Initial burner configuration:")
for i, (x, y) in enumerate(burners_init):
    print(f"- Burner {i + 1}: x = {x:.4f}, y = {y:.4f}")

# Solve the Poisson equation for the initial (random) burner configuration
P = solve_pressure_distribution(burners_init)

# Visualise the pressure distribution
plt.figure()
contour = plt.contourf(X, Y, P, levels=100, cmap="jet")
plt.colorbar(contour, label="Pressure (Pa)")

# Plot the burner locations
burners = np.array(burners_init)
plt.scatter(burners[:, 0], burners[:, 1], color="k", marker="x", label="Burners")
plt.legend()

# Plot settings
plt.title("Pressure distribution in the combustion chamber")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.tight_layout()
plt.show()

#### Optimised pressure distribution

The optimised pressure distribution results from adjusting burner locations and intensities to minimise pressure variance. This configuration achieves a more uniform pressure field, reducing localised high-pressure regions and ensuring a smoother gradient across the chamber.

In [ ]:
# Convert to flat format for optimisation
initial_condition = flatten_burners(burners_init)

# Define bounds: (x, y) in [0,1] for each burner
epsilon = 5e-2
bounds = ([(epsilon, Lx - epsilon), (epsilon, Ly - epsilon)]) * num_burners

# Run the gradient-based optimisation
result = minimize(
    objective_function,
    initial_condition,
    bounds=bounds,
    method="L-BFGS-B",
)

print(result)

# Extract the optimised burner configuration
burners_opt = unflatten_burners(result.x, num_burners)

In [ ]:
# Print the optimised burner configuration
print("Optimised burner configuration:")
for i, (x, y) in enumerate(burners_opt):
    print(f"- Burner {i + 1}: x = {x:.4f}, y = {y:.4f}")

# Solve the Poisson equation for the initial (random) burner configuration
P_opt = solve_pressure_distribution(burners_opt)

# Visualise the pressure distribution
plt.figure()
contour = plt.contourf(X, Y, P_opt, levels=100, cmap="jet")
plt.colorbar(contour, label="Pressure (Pa)")

# Plot the burner locations
burners = np.array(burners_opt)
plt.scatter(
    burners[:, 0], burners[:, 1], color="k", marker="x", label="Optimised burners"
)
plt.legend()

# Plot settings
plt.title("Pressure distribution in the combustion chamber")
plt.xlabel("x (m)")
plt.ylabel("y (m)")
plt.tight_layout()
plt.show()